In [1]:
import numpy as np
import pandas as pd

### 1. What is the difference between number of `runs` and number of `loops` in `%%timeit` command output?

In [2]:
%%timeit -n 10 -r 3
sum([k for k in range(1_000_000)])

53.7 ms ± 769 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


According to https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit :
```
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
```
Code will be executed `r` number of times with `n` loops in each run eg.
```python
%%timeit -n 10 -r 3
sum([k for k in range(1_000_000)])
```
will execute the code 3\*10=30 times

I don't know real purpose of giving in this case two parameters instead of just one. I guess it is probably to provide good user experience.

### 2. What is the difference in NumPy between double fancy indexing and mixed indexing?

Let's consider the example below:

In [3]:
col_indices = [0,1,2]

In [4]:
array = np.array([x for x in range(9)]).reshape(3, -1)
array

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [5]:
# case1 - mixed indexing
array[:, col_indices]

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [6]:
# case2 - double fancy indexing
array[[0,1,2], col_indices]

array([0, 4, 8])

To see the difference let's take a little longer indexing:

In [7]:
# you need to remember that double fancy indexing 
# is like a Battleships (or Sea Battle) game
# let's consider the following case:
array[[0,0,0,1,2,2,1,2], [2,1,2,1,0,0,1,1]]

array([2, 1, 2, 4, 6, 6, 4, 7])

In [8]:
# slicing using `:` or not using anythong in the columns slicing
# means `dear numpy - give me everything you got`
# which is a slightly different concept
array[[0,0,0,1,2,2,1,2], :]

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [3, 4, 5],
       [6, 7, 8],
       [6, 7, 8],
       [3, 4, 5],
       [6, 7, 8]])

In [9]:
array[:, [2,1,2,1,0,0,1,1]]

array([[2, 1, 2, 1, 0, 0, 1, 1],
       [5, 4, 5, 4, 3, 3, 4, 4],
       [8, 7, 8, 7, 6, 6, 7, 7]])

In [10]:
# in general fancy indexing sometimes may return slightly different 
# result than `standard` slicing, look at this:
array[0,1] # <<<< slicing

1

In [11]:
array[[0],[1]] #  <<<< fancy indexing

array([1])

### 3. How to divide NumPy arrays along given axis?

The only way I know is to use reshape to provide proper broadcastiong. I don't know any divide-like function with `axis` parameter. 

In [12]:
top = np.array([x for x in range(9)]).reshape(3, -1) # numerator
bot = np.array([x for x in range(1,4)]) # denominator

In [13]:
top

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [14]:
bot

array([1, 2, 3])

In [15]:
# first way is to use reshape
top/bot.reshape(-1, 3)

array([[0.        , 0.5       , 0.66666667],
       [3.        , 2.        , 1.66666667],
       [6.        , 3.5       , 2.66666667]])

In [16]:
top/bot.reshape(3, -1)

array([[0.        , 1.        , 2.        ],
       [1.5       , 2.        , 2.5       ],
       [2.        , 2.33333333, 2.66666667]])

There are many other functions to appply 'division' for NumPy arrays. I don't know any significant difference between them.

In [17]:
top.__truediv__(bot)

array([[0.        , 0.5       , 0.66666667],
       [3.        , 2.        , 1.66666667],
       [6.        , 3.5       , 2.66666667]])

In [18]:
np.true_divide(top, bot)

array([[0.        , 0.5       , 0.66666667],
       [3.        , 2.        , 1.66666667],
       [6.        , 3.5       , 2.66666667]])

In [19]:
np.divide(top, bot.reshape(3,1))

array([[0.        , 1.        , 2.        ],
       [1.5       , 2.        , 2.5       ],
       [2.        , 2.33333333, 2.66666667]])

### 4. What does the `copy` argument in `pd.Series`? 
Let's consider the following example

In [20]:
x = pd.Series([1.0])
y = pd.Series(x, copy=False)
z = pd.Series(x, copy=True)

x[0] = 2.0 # value of x is changed
print(x[0]) # you will see changed value of 2.0
print(y[0]) # you will see changed value of 2.0
print(z[0]) # you will not notice any change
            # because input data was copied (reference no longer exists)!

2.0
2.0
1.0
